# List of Fighter and Attack Aircraft 

This is a list of military aircraft that are primarily designed for air-to-air combat and ground attack. 

We will be using pandas, requests and beautifulsoup to webscrape the Fighter Aircraft and Attack Aircraft wikipedia pages.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# First wikipedia page to webscrape - List of Fighter Aircraft
page = requests.get('https://en.wikipedia.org/wiki/List_of_fighter_aircraft').text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_="wikitable sortable")

df = pd.read_html(str(table))
df = pd.concat(df)
df['Purpose'] = 'Fighter'
print(df)

                                         Type  Country                 Class  \
0                                    ACAZ C.2  Belgium      Two-seat fighter   
1                     Adamoli-Cattani fighter    Italy                   NaN   
2                                    AD Scout       UK  Zeppelin interceptor   
3     ADA/HAL Advanced Medium Combat Aircraft    India                   NaN   
4                       AEG D.I, D.II & D.III  Germany                   NaN   
...                                       ...      ...                   ...   
1302                   Yakovlev Yak-50 (1949)     USSR                   NaN   
1303                            Yatsenko I-28     USSR                   NaN   
1304                          Yokosuka D4Y2-S    Japan         Night fighter   
1305                     Yokosuka P1Y2 Kyokko    Japan         Night fighter   
1306            Zeppelin-Lindau (Dornier) D.I  Germany                   NaN   

      Date      Status  No.          No

In [2]:
# Second wikipedia page to webscrape - List of Ground Attack Aircraft
page = requests.get('https://en.wikipedia.org/wiki/List_of_attack_aircraft').text
soup = BeautifulSoup(page, 'html.parser')
table = soup.find('table', class_="wikitable sortable")

df2 = pd.read_html(str(table))
df2 = pd.concat(df2)
df2['Purpose'] = 'Attack'
print(df2)

                   Type  Country  Class  Date  Status   No.  Notes Purpose
0              AEG DJ.I  Germany    NaN  1918     NaN    1+    NaN  Attack
1               AEG J.I  Germany    NaN  1917     NaN   609    NaN  Attack
2      Aermacchi MB-326    Italy    NaN  1957     NaN   650    NaN  Attack
3      Aermacchi MB-339    Italy    NaN  1976     NaN  213+    NaN  Attack
4      Aermacchi SF.260    Italy    NaN  1964     NaN  860+    NaN  Attack
..                  ...      ...    ...   ...     ...   ...    ...     ...
249  Westland Whirlwind       UK    NaN  1938     NaN   116    NaN  Attack
250     Westland Wyvern       UK    NaN  1946     NaN   127    NaN  Attack
251           Xian JH-7    China    NaN  1988     NaN   240    NaN  Attack
252        Yokosuka B4Y    Japan    NaN  1935     NaN   205    NaN  Attack
253        Yokosuka D4Y    Japan    NaN  1940     NaN  2038    NaN  Attack

[254 rows x 8 columns]


### Reading data in pandas

In [3]:
# Read headers
df.columns

Index(['Type', 'Country', 'Class', 'Date', 'Status', 'No.', 'Notes',
       'Purpose'],
      dtype='object')

### Concat dataframes

In [4]:
frames = [df, df2]

result = pd.concat(frames)

result

,Type,Country,Class,Date,Status,No.,Notes,Purpose
0,ACAZ C.2,Belgium,Two-seat fighter,1926,Prototype,1,NaN,Fighter
1,Adamoli-Cattani fighter,Italy,NaN,1918,Prototype,1,NaN,Fighter
2,AD Scout,UK,Zeppelin interceptor,1915,Prototype,4,NaN,Fighter
3,ADA/HAL Advanced Medium Combat Aircraft,India,NaN,2019,Project,0,[1],Fighter
4,"AEG D.I, D.II & D.III",Germany,NaN,1917,Prototype,3,NaN,Fighter
...,...,...,...,...,...,...,...,...
249,Westland Whirlwind,UK,NaN,1938,NaN,116,NaN,Attack
250,Westland Wyvern,UK,NaN,1946,NaN,127,NaN,Attack
251,Xian JH-7,China,NaN,1988,NaN,240,NaN,Attack
252,Yokosuka B4Y,Japan,NaN,1935,NaN,205,NaN,Attack


### Make changes to the data

In [5]:
# Drop column
result = result.drop(columns=['Notes', 'Class', 'Status'])

result

,Type,Country,Date,No.,Purpose
0,ACAZ C.2,Belgium,1926,1,Fighter
1,Adamoli-Cattani fighter,Italy,1918,1,Fighter
2,AD Scout,UK,1915,4,Fighter
3,ADA/HAL Advanced Medium Combat Aircraft,India,2019,0,Fighter
4,"AEG D.I, D.II & D.III",Germany,1917,3,Fighter
...,...,...,...,...,...
249,Westland Whirlwind,UK,1938,116,Attack
250,Westland Wyvern,UK,1946,127,Attack
251,Xian JH-7,China,1988,240,Attack
252,Yokosuka B4Y,Japan,1935,205,Attack


### Filtering data

In [6]:
# Find Aircraft from the US
us_aircraft = result.loc[(result['Country'] == "US")]
us_aircraft

,Type,Country,Date,No.,Purpose
14,Aeromarine PG-1,US,1922,1,Fighter
37,Albree Pigeon-Fraser Pursuit,US,1917,3,Fighter
129,Bell YFM-1 Airacuda,US,1937,13,Fighter
130,Bell XFL Airabonita,US,1940,1,Fighter
131,Bell P-39 Airacobra,US,1938,9584,Fighter
...,...,...,...,...,...
242,Vought F4U Corsair,US,1940,12571,Attack
243,Vought SB2U Vindicator,US,1936,260,Attack
244,Vought SBU Corsair,US,1933,125,Attack
245,"Vultee V-11, V-12 and A-19",US,1935,224,Attack


### Remove special characters from Date column by extracting numbers 

An error will be encountered when trying to sort by Date, which means that this column may have special characters.

Upon further analysis, we see that multiple fields have question marks or parenthesis in them.

In [7]:
us_aircraft = us_aircraft.assign(Date = lambda x: x['Date'].str.extract('(\d+)'))

### Sorting data by Date

In [8]:
us_aircraft.sort_values(['Date'], ascending=[1], inplace=True)

us_aircraft.head(50)

,Type,Country,Date,No.,Purpose
32,Boeing GA-1,US,1920,10,Attack
55,Curtiss Falcon,US,1924,488,Attack
77,Douglas XA-2,US,1926,1,Attack
155,Martin BM,US,1929,35,Attack
96,Fokker XA-7,US,1931,1,Attack
54,Curtiss A-8,US,1931,13,Attack
244,Vought SBU Corsair,US,1933,125,Attack
52,Curtiss A-12 Shrike,US,1933,46,Attack
99,Great Lakes BG,US,1933,61,Attack
187,Northrop Gamma 2C & 2E,US,1933,52 c.,Attack


### Write the dataframe to parquet format.

After preprocessing data using Python, we can convert the dataframe to a format that can be loaded into a visualization tool (e.g., Microsoft PowerBI)

In [9]:
us_aircraft.to_parquet('us_aircraft.parquet', engine='fastparquet') # https://fastparquet.readthedocs.io/en/latest/
